# Import OpenAI Key

Here, we securely import the OpenAI API key. The `getpass` function is used so the key is hidden when entered.  
We store the key in an environment variable and then retrieve it to initialize the OpenAI API client.

In [1]:
import os
from getpass import getpass

# This way the key is hidden when typed
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [ ]:
import openai

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Please set OPENAI_API_KEY environment variable.")

client = openai.OpenAI(api_key=api_key)

# Run Prompts Through ChatGPT in Batches

This function takes a base message template and a list of prompts, then sends each prompt through ChatGPT as an independent session.

- `deepcopy` ensures we don't modify the original message template.
- Each prompt replaces the last user message in the template.
- The function calls the OpenAI API with the updated message history.
- If an error occurs (e.g., rate limit or malformed prompt), it prints the error and adds "ERROR" to the results.
- It returns a list of model-generated responses, one per prompt.


In [3]:
from copy import deepcopy
from openai import OpenAIError

def run_batch_messages_with_prompts(base_messages, prompts, model="gpt-4o-mini", temperature=0.0):
    responses = []

    for prompt in prompts:
        messages = deepcopy(base_messages)

        if messages[-1]['role'] == 'user':
            messages[-1]['content'] = prompt
        else:
            messages.append({"role": "user", "content": prompt})

        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature
            )
            reply = completion.choices[0].message.content.strip()
            responses.append(reply)
        except OpenAIError as e:
            print(f"Error with prompt: {prompt[:50]}... -> {e}")
            responses.append("ERROR")

    return responses

# Basic Classification Prompt (Single Example)

This prompt sets up a simple few-shot classification task for ChatGPT.

- The **system message** instructs the model to classify a short mental health-related text into one of 7 categories.
- The **user message** provides a sample input text that describes symptoms of depression.
- The **assistant message** shows the correct label, wrapped in `<answer>` tags, as expected in the output format.
- The final **user message** is a placeholder (`STATEMENT_GOES_HERE`) that will be replaced with actual input during evaluation.

This prompt teaches the model what kind of task it's doing, the output format, and gives it one labeled example to follow.


In [4]:
base_messages = [
    {
        "role": "system",
        "content": (
            "You are a mental health classification assistant. "
            "Given a short piece of text, your job is to classify it into one of the following mental health categories:\n\n"
            "Normal, Depression, Suicidal, Anxiety, Stress, Bipolar, Personality disorder.\n\n"
            "Respond only with the predicted class, wrapped in <answer> tags. Do not explain your reasoning."
        )
    },
    {
        "role": "user",
        "content": (
            "I haven’t felt like myself in months. I can’t sleep, I can’t eat, and I cry almost every day."
        )
    },
    {
        "role": "assistant",
        "content": "<answer>Depression</answer>"
    },
    {
        "role": "user",
        "content": "STATEMENT_GOES_HERE"
    }
]

# Chain-of-Thought (CoT) Classification Prompt

This prompt uses a Chain-of-Thought (CoT) format to help the model reason more effectively before making a classification.

- The **system message** instructs the model to first explain its reasoning by quoting relevant parts of the input, then provide a label wrapped in `<answer>` tags.
- The prompt includes one example for each of the 7 mental health categories: Normal, Depression, Suicidal, Anxiety, Stress, Bipolar, and Personality disorder.
- Each **user message** provides a sample input, and each **assistant message** explains the reasoning followed by the correct label.
- The final **user message** is a placeholder where the actual input will be inserted during evaluation.

This format encourages the model to be more thoughtful and transparent in its decision-making, which can improve accuracy—especially for nuanced or overlapping mental health conditions.


In [5]:
base_messages_CoT = [
    {
        "role": "system",
        "content": (
            "You are a mental health classification assistant. "
            "Given a short piece of text, your job is to classify it into one of the following mental health categories:\n\n"
            "Normal, Depression, Suicidal, Anxiety, Stress, Bipolar, Personality disorder.\n\n"
            "First, briefly explain your reasoning by quoting relevant parts of the input. "
            "Then, on a new line, respond with the predicted class, wrapped in <answer> tags."
        )
    },
    # --- Normal ---
    {
        "role": "user",
        "content": "I'm feeling fine, just a bit tired after work. Nothing really bothering me today."
    },
    {
        "role": "assistant",
        "content": (
            "The statement says 'I'm feeling fine' and 'Nothing really bothering me today', which indicates no signs of mental health distress.\n"
            "<answer>Normal</answer>"
        )
    },
    # --- Depression ---
    {
        "role": "user",
        "content": "I haven’t felt like myself in months. I can’t sleep, I can’t eat, and I cry almost every day."
    },
    {
        "role": "assistant",
        "content": (
            "The user reports prolonged negative symptoms like 'I can’t sleep', 'I can’t eat', and 'cry almost every day', which are classic indicators of depression.\n"
            "<answer>Depression</answer>"
        )
    },
    # --- Suicidal ---
    {
        "role": "user",
        "content": "I don’t want to live anymore. I’m tired of pretending everything is okay."
    },
    {
        "role": "assistant",
        "content": (
            "The phrase 'I don’t want to live anymore' clearly indicates suicidal ideation.\n"
            "<answer>Suicidal</answer>"
        )
    },
    # --- Anxiety ---
    {
        "role": "user",
        "content": "My heart races constantly and I worry about everything. Even small tasks feel overwhelming."
    },
    {
        "role": "assistant",
        "content": (
            "The user says 'my heart races constantly' and 'I worry about everything', which are strong indicators of anxiety.\n"
            "<answer>Anxiety</answer>"
        )
    },
    # --- Stress ---
    {
        "role": "user",
        "content": "Work has been insane lately. Deadlines, meetings, and barely any sleep."
    },
    {
        "role": "assistant",
        "content": (
            "The statement reflects situational overwhelm with 'Deadlines, meetings, and barely any sleep', which points to stress.\n"
            "<answer>Stress</answer>"
        )
    },
    # --- Bipolar ---
    {
        "role": "user",
        "content": "Some days I feel unstoppable, like I can do anything. Other days I can’t get out of bed."
    },
    {
        "role": "assistant",
        "content": (
            "The alternating mood between 'unstoppable' and 'can’t get out of bed' suggests mood swings typical of bipolar disorder.\n"
            "<answer>Bipolar</answer>"
        )
    },
    # --- Personality disorder ---
    {
        "role": "user",
        "content": "I push people away even though I want them close. I don't know who I really am."
    },
    {
        "role": "assistant",
        "content": (
            "The statement 'I push people away even though I want them close' and 'I don't know who I really am' reflect identity disturbance and interpersonal struggles, which are hallmarks of personality disorders.\n"
            "<answer>Personality disorder</answer>"
        )
    },
    # --- The actual input prompt ---
    {
        "role": "user",
        "content": "STATEMENT_GOES_HERE"
    }
]


# Targeted Stress Classification Prompt (with Contrastive Reasoning)

This prompt is designed to help the model better recognize and distinguish the "Stress" label.

- The **system message** includes clear, concise definitions of all 7 mental health categories, with extra emphasis on what distinguishes Stress from others.
- The **user message** describes a realistic situation involving work-related pressure and physical tension — symptoms consistent with Stress.
- The **assistant message** uses Chain-of-Thought reasoning to justify the "Stress" label using direct quotes, and then explicitly explains why each of the other 6 categories does not apply.
- The final label is wrapped in `<answer>` tags for consistency with the rest of the evaluation pipeline.

This format is intended to teach the model a deeper understanding of Stress by showing not just why it fits, but why other conditions do not — helping reduce false positives and confusion with overlapping categories like Anxiety or Depression.


In [20]:
base_messages_stress_example = [
    {
        "role": "system",
        "content": (
            "You are a mental health classification assistant. "
            "Given a short statement, your task is to classify it into one or more of the following categories:\n\n"
            "Normal: The person does not express any significant distress, mental illness, or dysfunction.\n"
            "Depression: Persistent sadness, hopelessness, lack of motivation, low energy, or loss of interest in life.\n"
            "Suicidal: Expressing desire or intent to harm oneself or end one's life.\n"
            "Anxiety: Excessive worry, fear, or physiological symptoms like a racing heart, restlessness, or panic.\n"
            "Stress: A response to external pressures or demands, often situational (e.g., deadlines, workload), usually without deeper psychological symptoms.\n"
            "Bipolar: Mood swings including both depressive episodes and periods of elevated or irritable mood (mania/hypomania).\n"
            "Personality disorder: Persistent patterns of unstable emotions, distorted self-image, and difficulty in interpersonal relationships.\n\n"
            "Respond by quoting key parts of the statement to justify the correct label. If applicable, explain why other labels do not fit. "
            "Conclude with your final label wrapped in <answer> tags."
        )
    },
    {
        "role": "user",
        "content": (
            "I’ve been buried in deadlines all week. My manager keeps piling more on me. "
            "I haven’t been sleeping well and I feel tense all the time, but I know I just have to get through this project."
        )
    },
    {
        "role": "assistant",
        "content": (
            "The statement mentions being 'buried in deadlines all week' and a manager 'piling more on', which indicates external pressures. "
            "The person says they 'haven’t been sleeping well' and 'feel tense all the time', which are physical and emotional signs of strain.\n\n"
            "This aligns with the definition of **Stress**: a response to situational pressures like workload, without signs of deeper psychological conditions.\n\n"
            "- **Not Depression**: There’s no mention of hopelessness, sadness, or lack of motivation.\n"
            "- **Not Suicidal**: The person expresses no desire or intent to harm themselves.\n"
            "- **Not Anxiety**: While they feel tense, there is no mention of excessive fear or uncontrollable worry.\n"
            "- **Not Bipolar**: There are no mood swings or episodes of high energy.\n"
            "- **Not Personality disorder**: There are no signs of unstable identity or relationship difficulties.\n"
            "- **Not Normal**: The person is clearly distressed by workload and sleep disruption, so this isn't a neutral or unaffected state.\n\n"
            "<answer>Stress</answer>"
        )
    },
    {
        "role": "user",
        "content": "STATEMENT_GOES_HERE"
    }
]


# Patrick Bateman Example

This example demonstrates how the model uses in-context learning and chain-of-thought (CoT) reasoning
to analyze a monologue from *American Psycho*. By quoting relevant parts of the text and eliminating
other possibilities, the model correctly identifies **Personality disorder** as the most fitting label.


In [35]:
import textwrap

# Patrick Bateman monologue prompt
prompt = (
    "My name is Patrick Bateman. I’m 27 years old. I believe in taking care of myself, and a balanced diet and a rigorous exercise routine.\n"
    "In the morning, if my face is a little puffy, I’ll put on an ice pack while doing my stomach crunches. I can do a thousand now.\n"
    "After I remove the ice pack I use a deep pore cleanser lotion. In the shower I use a water activated gel cleanser,\n"
    "then a honey almond body scrub, and on the face an exfoliating gel scrub.\n"
    "Then I apply an herb-mint facial masque which I leave on for 10 minutes while I prepare the rest of my routine.\n"
    "I always use an after shave lotion with little or no alcohol, because alcohol dries your face out and makes you look older.\n"
    "Then moisturizer, then an anti-aging eye balm followed by a final moisturizing protective lotion."
)

# Run it through existing function
response = run_batch_messages_with_prompts(base_messages_CoT, [prompt])[0]

# Print the entire conversation
print("=== Conversation ===\n")

# Print the inserted prompt
print("User:", prompt, "\n")

# Wrap the assistant output to 100 characters per line
print("Assistant:\n" + textwrap.fill(response, width=100))


=== Conversation ===

User: My name is Patrick Bateman. I’m 27 years old. I believe in taking care of myself, and a balanced diet and a rigorous exercise routine.
In the morning, if my face is a little puffy, I’ll put on an ice pack while doing my stomach crunches. I can do a thousand now.
After I remove the ice pack I use a deep pore cleanser lotion. In the shower I use a water activated gel cleanser,
then a honey almond body scrub, and on the face an exfoliating gel scrub.
Then I apply an herb-mint facial masque which I leave on for 10 minutes while I prepare the rest of my routine.
I always use an after shave lotion with little or no alcohol, because alcohol dries your face out and makes you look older.
Then moisturizer, then an anti-aging eye balm followed by a final moisturizing protective lotion. 

Assistant:
The text reflects an excessive focus on appearance and self-care routines, which can indicate traits
associated with narcissistic personality disorder. The meticulous detail

# Load and Prepare Mental Health Dataset

This section handles downloading, processing, and loading a sample of the mental health dataset for evaluation.

- We use `kagglehub` to download the dataset from Kaggle and move it to a working directory in Colab.
- The dataset is read using `pandas`, and we display all the unique labels in the `status` column.
- We randomly sample 100 examples to create a manageable subset for testing and save it to a CSV file.
- A custom PyTorch `Dataset` class (`MentalHealthDataset`) is defined to load the sample data.
- Finally, we create a `DataLoader` to batch the data during evaluation.

This setup allows us to feed real mental health statements and their labels into our classification pipeline for testing and analysis.


In [7]:
import kagglehub
import shutil
import os

# Download dataset (goes to ~/.cache/kagglehub by default)
path = kagglehub.dataset_download("suchintikasarkar/sentiment-analysis-for-mental-health")

# Define where you want to move it
destination = "/content/mental_health_dataset"

# Move the dataset to /content
if not os.path.exists(destination):
    shutil.copytree(path, destination)

print("Moved dataset to:", destination)

Moved dataset to: /content/mental_health_dataset


In [8]:
import pandas as pd

# Load the CSV
df = pd.read_csv('/content/mental_health_dataset/Combined Data.csv')

# Show unique labels in the 'status' column
labels = sorted(df['status'].dropna().unique())
print("Unique labels found in 'status' column:\n")
for label in labels:
    print(f"- {label}")

Unique labels found in 'status' column:

- Anxiety
- Bipolar
- Depression
- Normal
- Personality disorder
- Stress
- Suicidal


In [9]:
# Random sample of 100 rows
sample_df = df.sample(n=100, random_state=42).reset_index(drop=True)

In [10]:
sample_df.to_csv('/content/mental_health_dataset/sample_100.csv', index=False)

In [2]:
import pandas as pd
from torch.utils.data import Dataset

class MentalHealthDataset(Dataset):
    def __init__(self, csv_path):
        self.data = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        return {
            "text": row["statement"],
            "label": row.get("status", None)
        }

In [11]:
from torch.utils.data import DataLoader

dataset = MentalHealthDataset('/content/mental_health_dataset/sample_100.csv')
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

# Evaluate Model Predictions

This section defines the evaluation logic for running the classification model over the dataset.

- `evaluate_model()` takes a PyTorch `DataLoader`, a base message template, and optional model settings (model name, temperature).
- For each batch, it extracts the input texts and labels, sends the prompts through the `run_batch_messages_with_prompts()` function, and collects the responses.
- The `extract_answer_tag()` function is used to parse the predicted label from each response.
  - It looks for content inside `<answer>...</answer>` tags.
  - If multiple labels are returned (comma-separated), it selects the **first one** as the model’s primary classification.
- Predictions and ground truth labels are stored and returned for further evaluation.

This evaluation pipeline helps measure how well the model performs across various label categories.


In [12]:
import re
from tqdm import tqdm

def evaluate_model(dataloader, base_messages, model="gpt-4o-mini", temperature=0.0):
    all_preds = []
    all_labels = []

    for batch in tqdm(dataloader, desc="Evaluating"):
        prompts = batch["text"]
        labels = batch.get("label", [None] * len(prompts))  # handles unlabeled test data

        # Get model responses
        responses = run_batch_messages_with_prompts(
            base_messages=base_messages,
            prompts=prompts,
            model=model,
            temperature=temperature
        )

        # Extract the answer between <answer>...</answer>
        preds = [
            extract_answer_tag(response) for response in responses
        ]

        all_preds.extend(preds)
        all_labels.extend(labels)

    return all_preds, all_labels


def extract_answer_tag(text):
    match = re.search(r"<answer>(.*?)</answer>", text, re.IGNORECASE | re.DOTALL)
    if match:
        answer_content = match.group(1).strip()
        # Handle comma-separated multi-label case
        first_label = answer_content.split(",")[0].strip()
        return first_label
    return "UNKNOWN"


# Generate and Compare Classification Reports

This section evaluates the model using three different prompt formats:

1. `base_messages` – A basic few-shot prompt with no reasoning.
2. `base_messages_CoT` – A Chain-of-Thought prompt that includes explanations and examples for each class.
3. `base_messages_stress_example` – A targeted prompt to help the model better distinguish the "Stress" category.

For each case:
- We call `evaluate_model()` to get the model's predictions.
- We use `classification_report()` from scikit-learn to print precision, recall, and F1-score for each label.

This comparison helps us analyze which prompt design yields the best overall performance and where the model needs improvement.


In [16]:
from sklearn.metrics import classification_report

# Run evaluation
preds, labels = evaluate_model(dataloader, base_messages)

# Print classification report
print("\n", classification_report(labels, preds, digits=3))

Evaluating: 100%|██████████| 13/13 [01:16<00:00,  5.90s/it]


                       precision    recall  f1-score   support

             Anxiety      0.300     0.750     0.429         4
             Bipolar      0.250     0.500     0.333         2
          Depression      0.923     0.333     0.490        36
              Normal      0.926     0.806     0.862        31
Personality disorder      0.250     1.000     0.400         2
              Stress      0.167     0.200     0.182         5
            Suicidal      0.562     0.900     0.692        20

            accuracy                          0.620       100
           macro avg      0.483     0.641     0.484       100
        weighted avg      0.762     0.620     0.623       100



In [18]:
# Run evaluation
preds, labels = evaluate_model(dataloader, base_messages_CoT)

# Print classification report
print("\n", classification_report(labels, preds, digits=3))

Evaluating: 100%|██████████| 13/13 [03:07<00:00, 14.41s/it]


                       precision    recall  f1-score   support

             Anxiety      0.333     0.750     0.462         4
             Bipolar      1.000     0.500     0.667         2
          Depression      0.882     0.417     0.566        36
              Normal      0.903     0.903     0.903        31
Personality disorder      0.200     1.000     0.333         2
              Stress      0.250     0.200     0.222         5
            Suicidal      0.630     0.850     0.723        20
              Trauma      0.000     0.000     0.000         0

            accuracy                          0.670       100
           macro avg      0.525     0.577     0.485       100
        weighted avg      0.773     0.670     0.678       100




/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Run evaluation
preds, labels = evaluate_model(dataloader, base_messages_stress_example)

# Print classification report
print("\n", classification_report(labels, preds, digits=3))


                       precision    recall  f1-score   support

             Anxiety      0.429     0.750     0.545         4
             Bipolar      1.000     0.500     0.667         2
          Depression      0.828     0.667     0.738        36
              Normal      0.875     0.903     0.889        31
Personality disorder      0.250     0.500     0.333         2
              Stress      0.000     0.000     0.000         5
            Suicidal      0.615     0.800     0.696        20

            accuracy                          0.730       100
           macro avg      0.571     0.589     0.553       100
        weighted avg      0.734     0.730     0.722       100

